# Работа с Excel

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [10]:
import pandas as pd

fname = 'recipes_sample.csv'
recipes = pd.read_csv(fname, delimiter = ',', parse_dates=['submitted'])

fname_2 = 'reviews_sample.csv'
reviews = pd.read_csv(fname_2, delimiter = ',', index_col = 'Unnamed: 0')

reviews.reset_index(drop= True , inplace= True)
recipes = recipes.drop(columns=['contributor_id', 'n_steps'])

print(recipes)
print(reviews)

                                               name      id  minutes  \
0             george s at the cove  black bean soup   44123       90   
1                healthy for them  yogurt popsicles   67664       10   
2                      i can t believe it s spinach   38798       30   
3                              italian  gut busters   35173       45   
4          love is in the air  beef fondue   sauces   84797       25   
...                                             ...     ...      ...   
29995  zurie s holey rustic olive and cheddar bread  267661       80   
29996          zwetschgenkuchen  bavarian plum cake  386977      240   
29997   zwiebelkuchen   southwest german onion cake  103312       75   
29998                                   zydeco soup  486161       60   
29999        cookies by design   cookies on a stick  298512       29   

       submitted                                        description  \
0     2002-10-25  an original recipe created by chef scott meska

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [15]:
import pandas as pd
import random

reviews_fс = reviews.sample(frac=0.05)
recipes_fс = recipes.sample(frac=0.05)

writer = pd.ExcelWriter('recipes.xlsx')

reviews_fс.to_excel(writer, sheet_name='Отзывы')
recipes_fс.to_excel(writer, sheet_name='Рецепты')

writer.save()

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [ ]:
import xlwings as xw
import pandas as pd

file_name = 'recipes.xlsx'
wb = xw.Book(file_name)
recipes_df = wb.sheets['Рецепты'].used_range.options(pd.DataFrame, index=False, header=True).value

recipes_df['seconds_assign'] = recipes_df['minutes'] * recipes_df['step_duration_seconds']

wb.sheets['Рецепты'].range('F1').value = 'seconds_assign'
wb.sheets['Рецепты'].range('F2').options(transpose=True).value = recipes_df['seconds_assign']

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [ ]:
import xlwings as xw

wb = xw.Book('recipes.xlsx')
sht = wb.sheets['Рецепты']

nrows = sht.range('A1').current_region.last_cell.row

sht.range('I1').value = 'seconds_formula'
for i in range(2, nrows+1):
    sht.range('I{}'.format(i)).formula = '=E{}*60'.format(i)

wb.save()

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
import xlwings as xw

wb = xw.Book('recipes.xlsx')
sht = wb.sheets['Рецепты']

sht.range('C1:C2').api.Select()
sht.range('C1:C2').api.Font.Bold = True
sht.range('C1:C2').api.HorizontalAlignment = -4108

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:
import xlwings as xw

book = xw.Book('recipes.xlsx')
sheet = book.sheets['Рецепты']

minutes_column = sheet.range('E2:E1001')

green_color = (0, 255, 0)
yellow_color = (255, 255, 0)
red_color = (255, 0, 0)
color_scale_rules = [(0, green_color), (5, yellow_color), (10, red_color)]

minutes_column.color_scale(*color_scale_rules)

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
import xlwings as xw

def validate():
    wb = xw.Book('recipes.xlsx')
    recipes_sheet = wb.sheets['Рецепты']
    reviews_sheet = wb.sheets['Отзывы']
    
    recipe_id_col = recipes_sheet.range('A1').expand('table').value.index('recipe_id')
    review_recipe_id_col = reviews_sheet.range('A1').expand('table').value.index('recipe_id')
    rating_col = reviews_sheet.range('A1').expand('table').value.index('rating')
    
    n_reviews_dict = {}
    for row in reviews_sheet.range('A2').expand('table').value:
        recipe_id = row[review_recipe_id_col]
        if recipe_id not in n_reviews_dict:
            n_reviews_dict[recipe_id] = 1
        else:
            n_reviews_dict[recipe_id] += 1
    
    for row in reviews_sheet.range('A2').expand('table'):
        recipe_id = row[review_recipe_id_col]
        rating = row[rating_col]
        
        if not (0 <= rating <= 5 and recipe_id in n_reviews_dict):
            row.color = (255, 0, 0)


9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 